In [ ]:
!pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 24.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.9 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=d3e464adb5f300a78261a5e2f7c36c44f22dfc05c633d291e9cd3154d96c3cec
  Stored in directory: /root/.cache/pip/wheels/f2/ed/dd/d3a556ad245ef9dc570c6bcd2f22886d17b0b408dd3bbb9ac3
Successfully built validators


In [ ]:
%%writefile app2.py

import streamlit as st
from PIL import Image , ImageOps
import numpy as np
from keras.preprocessing import image
from keras.models import load_model
import cv2

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Load the model
model = load_model('/content/drive/MyDrive/BrainCancerModel.h5')

st.write("# Cancer Detection")

# File upload
file = st.file_uploader("Please upload a brain scan file", type=["jpg", "png"])

# Define a function to process the image and make predictions
def import_and_predict(image_data, model):
    size = (150, 150)    
    image = ImageOps.fit(image_data, size, Image.ANTIALIAS)
    image = np.asarray(image)
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img_resize = (cv2.resize(img, dsize=(75, 75), interpolation=cv2.INTER_CUBIC)) / 255.
    img_reshape = img[np.newaxis, ...]
    pred = (model.predict(img_reshape) > 0.5).astype("int32")
    return pred

if file is None:
    st.text("Please upload an image file")
else:
    image = Image.open(file)
    st.image(image, use_column_width=True)
    predictions = import_and_predict(image, model)
    # st.write(predictions)
    
    if predictions == 0:
        st.write('<span style="font-size: 30px; font-weight: bold; text-align: center;">Cancer - Negative</span>', unsafe_allow_html=True)
    else:
        st.write('<span style="font-size: 30px; font-weight: bold; text-align: center;">Cancer - Positive</span>', unsafe_allow_html=True)
        if st.button("Proceed with Treatment"):
            with st.spinner("Loading..."):

                # Perform further treatment based on the positive prediction
                
                treatment_model = load_model('/content/drive/MyDrive/BrainCancerTreatment.h5')

                # Define the class labels
                class_labels = ['Mild', 'Moderate', 'Severe']

                # Function to preprocess the input image
                def preprocess_image(img):
                    img = img.resize((256, 256))  # Resize the image
                    img_array = np.array(img)
                    img_array = np.expand_dims(img_array, axis=0)
                    preprocessed_img = img_array / 255.0  # Normalize pixel values
                    return preprocessed_img

                # Function to predict the severity class
                def predict_severity(img):
                    preprocessed_img = preprocess_image(img)
                    prediction = treatment_model.predict(preprocessed_img)
                    predicted_class = np.argmax(prediction)
                    severity_class = class_labels[predicted_class]
                    return severity_class

                # Example usage
                img = Image.open(file)
                predicted_severity = predict_severity(img)
                st.write('<span style="font-size: 30px; font-weight: bold;">Predicted Severity: <span style="font-weight: normal; font-size: inherit;">{}</span></span>'.format(predicted_severity), unsafe_allow_html=True)

                # Custom CSS styling
                st.markdown(
                    """
                    <style>
                    .title {
                        color: #336699;
                        font-size: 28px;
                        font-weight: bold;
                        margin-bottom: 20px;
                    }
                    .paragraph {
                        color: #555555;
                        font-size: 16px;
                        line-height: 1.5;
                        margin-bottom: 10px;
                    }
                    .bold-text {
                        font-weight: bold;
                    }
                    </style>
                    """,
                    unsafe_allow_html=True
                )

                # Streamlit UI
                st.title("Brain Cancer Treatment Options")

                if(predicted_severity == "Mild"):
                    st.markdown('<div class="title">Mild Severity:</div>', unsafe_allow_html=True)
                    st.markdown('<div class="paragraph"><span class="bold-text">Surgery:</span> Surgical removal of the tumor is often the primary treatment for brain cancer. It aims to remove as much of the tumor as possible without causing damage to healthy brain tissue.</div>', unsafe_allow_html=True)
                    st.markdown('<div class="paragraph"><span class="bold-text">Radiation Therapy:</span> This treatment uses high-energy X-rays or other types of radiation to destroy cancer cells or prevent their growth.</div>', unsafe_allow_html=True)
                    st.markdown('<div class="paragraph"><span class="bold-text">Chemotherapy:</span> Medications are used to kill cancer cells or stop them from dividing. Chemotherapy may be administered orally or intravenously.</div>', unsafe_allow_html=True)

                elif(predicted_severity == "Moderate"):
                    st.markdown('<div class="title">Moderate Severity:</div>', unsafe_allow_html=True)
                    st.markdown('<div class="paragraph"><span class="bold-text">Surgery:</span> As with mild severity, surgery may be performed to remove the tumor.</div>', unsafe_allow_html=True)
                    st.markdown('<div class="paragraph"><span class="bold-text">Radiation Therapy:</span> It may be used after surgery to target any remaining cancer cells or as a standalone treatment to slow down tumor growth and relieve symptoms.</div>', unsafe_allow_html=True)
                    st.markdown('<div class="paragraph"><span class="bold-text">Chemotherapy:</span> In addition to surgery and radiation, chemotherapy may be used to target cancer cells throughout the body. It can be given orally or intravenously.</div>', unsafe_allow_html=True)
                
                elif(predicted_severity == "Severe"):
                    st.markdown('<div class="title">Severe Severity:</div>', unsafe_allow_html=True)
                    st.markdown('<div class="paragraph"><span class="bold-text">Surgery:</span> In severe cases, surgery may be more challenging, as the tumor may be larger or located in critical areas of the brain. Surgeons will aim to remove as much of the tumor as possible to relieve pressure and symptoms.</div>', unsafe_allow_html=True)
                    st.markdown('<div class="paragraph"><span class="bold-text">Radiation Therapy:</span> High-dose radiation therapy may be used to shrink the tumor and alleviate symptoms.</div>', unsafe_allow_html=True)
                    st.markdown('<div class="paragraph"><span class="bold-text">Chemotherapy:</span> More aggressive chemotherapy regimens may be employed to target cancer cells throughout the body. It can also be used in combination with radiation therapy to enhance treatment effectiveness.</div>', unsafe_allow_html=True)

                            

Writing app2.py


In [ ]:
!streamlit run app2.py &npx localtunnel --port 8501

[#.................] / fetchMetadata: sill resolveWithNewModule follow-redirect


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.86.247.162:8501

npx: installed 22 in 4.378s
your url is: https://cold-wasps-count.loca.lt
2023-06-01 14:42:29.633236: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 14:42:31.020795: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
1/1 [==============================] - 0s 77ms/step


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import streamlit as st
import tensorflow as tf
from keras.models import load_model
from tensorflow.keras.utils import load_img, img_to_array
from PIL import Image, ImageOps
import cv2
import numpy as np

# Load the model
model = load_model('/content/drive/MyDrive/BrainCancerModel (1).h5')

st.write("# Cancer Detection")

# File upload
file = st.file_uploader("Please upload a brain scan file", type=["jpg", "png"])

# Define a function to process the image and make predictions
def import_and_predict(image_data, model):
    size = (150, 150)    
    image = ImageOps.fit(image_data, size, Image.ANTIALIAS)
    image = np.asarray(image)
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img_resize = (cv2.resize(img, dsize=(75, 75), interpolation=cv2.INTER_CUBIC)) / 255.
    img_reshape = img[np.newaxis, ...]
    pred = (model.predict(img_reshape) > 0.5).astype("int32")
    return pred

if file is None:
    st.text("Please upload an image file")
else:
    image = Image.open(file)
    st.image(image, use_column_width=True)
    predictions = import_and_predict(image, model)
    st.write(predictions)
    
    if predictions == 0:
        st.write("Cancer - Negative")
    else:
         st.write("Cancer - Positive")
         if st.button("Proceed with Treatment"):
            # Load the treatment model
            treatment_model = load_model('/content/drive/MyDrive/BrainCancerTreatment.h5')
            

            # Perform the treatment based on the positive prediction
            st.write("Treatment in progress...")
            image_path = file.name
            class_labels = ['Mild', 'Moderate', 'Severe']

            # Function to preprocess the input image
            def preprocess_image(image_path):
              img = image.load_img(image_path, target_size=(256, 256))
              img_array = image.img_to_array(img)
              img_array = np.expand_dims(img_array, axis=0)
              preprocessed_img = img_array / 255.0  # Normalize pixel values
              return preprocessed_img


            # Add your treatment logic using the treatment_model
            def predict_severity(image_path):
              preprocessed_img = preprocess_image(image_path)
              prediction = model.predict(preprocessed_img)
              predicted_class = np.argmax(prediction)
              severity_class = class_labels[predicted_class]
              return severity_class
            predicted_severity = predict_severity(image_path)
            print("Predicted Severity:", predicted_severity)